<a href="https://colab.research.google.com/github/samko5sam/programming-language-class/blob/main/0227_GSheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0227 GSheets

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [11]:
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1dq45-lorkinokZVw7uxSLDfW2VP-wdMZTcMtlhCgRqg/edit?usp=sharing')

In [17]:
food_calories = {
    "grains": {
        "Rice": 200,
        "Noodles": 220,
        "Oatmeal": 150,
        "Sweet Potato": 90,
        "Corn": 100,
        "Whole Wheat Bread": 120
    },
    "proteins": {
        "Egg": 70,
        "Tofu": 80,
        "Chicken Breast": 165,
        "Fish": 140,
        "Shrimp": 90,
        "Pork": 210,
        "Beans": 130
    },
}

# Extract keys
grains = list(food_calories["grains"].keys())
proteins = list(food_calories["proteins"].keys())

# Find out the maximum length of the two lists
max_length = max(len(grains), len(proteins))

# Pad the shorter list with None
if len(grains) < max_length:
    grains += [" "] * (max_length - len(grains))
if len(proteins) < max_length:
    proteins += [" "] * (max_length - len(proteins))

# Create a DataFrame with paired items
df = pd.DataFrame({'grains': grains, 'proteins': proteins})
df

,grains,proteins
0,Rice,Egg
1,Noodles,Tofu
2,Oatmeal,Chicken Breast
3,Sweet Potato,Fish
4,Corn,Shrimp
5,Whole Wheat Bread,Pork
6,,Beans


In [19]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）
worksheet = gsheets.get_worksheet(0)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet, df)

In [27]:
# @title 讀取工作表2整理後寫入工作表3
worksheet2 = gsheets.get_worksheet(1)

# 取得 Google Sheets 內容
data = worksheet2.get_all_values()  # 取得所有儲存格資料，回傳為 2D 陣列

# 將資料轉換為 DataFrame
df_money = pd.DataFrame(data[1:], columns=data[0])
df_money

,除權息年度,股利,除息日,發放日,除息前股價,填息天數,年股利,年殖利率(%)
0,2025,2.7,2025/1/17,2025/2/20,198.05,4,2.7,1.36
1,2024,1,2024/7/16,2024/8/9,196.7,95,4,2.44
2,2024,3,2024/1/17,2024/2/21,131.65,3,4,2.44
3,2023,1.9,2023/7/18,2023/8/11,132,127,4.5,3.6
4,2023,2.6,2023/1/30,2023/3/7,118.1,2,4.5,3.6
5,2022,1.8,2022/7/18,2022/8/19,115.5,4,5,3.78
6,2022,3.2,2022/1/21,2022/3/4,149.1,4,5,3.78
7,2021,0.35,2021/7/21,2021/8/24,137.2,2,3.4,2.43
8,2021,3.05,2021/1/22,2021/3/9,143,27,3.4,2.43
9,2020,0.7,2020/7/21,2020/8/24,97.05,2,3.6,3.81


In [28]:
# Get unique entries based on '除權息年度'

# Grouping by '除權息年度' and selecting the first occurrence of '年股利' and '年殖利率(%)'
unique_df = df_money.groupby('除權息年度').agg({'年股利': 'first', '年殖利率(%)': 'first'}).reset_index()

# Sort values by '除權息年度' in descending order
unique_df = unique_df.sort_values(by='除權息年度', ascending=False)

unique_df

,除權息年度,年股利,年殖利率(%)
17,2025,2.7,1.36
16,2024,4,2.44
15,2023,4.5,3.6
14,2022,5,3.78
13,2021,3.4,2.43
12,2020,3.6,3.81
11,2019,3,3.77
10,2018,2.9,3.37
9,2017,2.4,3.09
8,2016,0.85,1.22


In [30]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）
worksheet3 = gsheets.get_worksheet(2)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet3, unique_df)